# Parallelization and Performance Optimization on Face Detection

In [ ]:
pip install pillow

In [1]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 45.5 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.2.25 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename, quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

**Parallel Execution**

In [ ]:
import scipy.misc

In [ ]:
import os
import concurrent.futures
import Time   
#import imageio
import time
import cv2 as cv
import multiprocessing
import numpy as np
from numpy import array
from scipy import misc
import datetime
import sys
from scipy.misc import toimage
from scipy.misc.pilutil import imread
from scipy.linalg import norm
from scipy import sum, average
num_cores = multiprocessing.cpu_count()
print(num_cores)
filename = take_photo("123.jpg")
print('Saved to {}'.format(filename))
img = cv.imread('123.jpg')

face = misc.imread('123.jpg', cv.IMREAD_UNCHANGED)
print (face.shape)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('123.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
(x, y, w, h)=(0,0,0,0)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for (x, y, w, h) in faces:
        cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('face.jpg', frame)

time1=time.time()

img = cv.imread("face.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(H):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            for j in range(W):
                gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('facegray.jpg', gray)

filename = take_photo("2nd.jpg")
print('Saved to {}'.format(filename))
img = cv.imread('2nd.jpg')
a = datetime.datetime.now()

face = misc.imread('2nd.jpg', cv.IMREAD_UNCHANGED)
print (face.shape)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('2nd.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for (x, y, w, h) in faces:
        cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('FaceinDataBase.jpg', frame)

img = cv.imread("FaceinDataBase.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(H):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            for j in range(W):
                gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('FaceinDataBasegray.jpg', gray)
cv.waitKey()

def normalize(arr):
  rng = arr.max()-arr.min()
  amin = arr.min()
  return (arr-amin)*255/rng

def compare_images(img1, img2):
  img1 = normalize(img1)
  img2 = normalize(img2)
  diff = img1 - img2 
  m_norm = sum(abs(diff))
  z_norm = norm(diff.ravel(), 0) 
  return (m_norm, z_norm)

def compareimagepercntage(n_m,img1):
  return n_m/img1.size*100<27.5
img1=cv.imread("FaceinDataBasegray.jpg")
img2=cv.imread("facegray.jpg")
x=len(np.array(img1))
y=len(np.array(img2))

width = int(img1.shape[1]*y/x)
height = int(img1.shape[0]*y/x)
dim = (width, height)
# resize image
resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
cv.imwrite('NewSizedImageGray.jpg',resized) 

img1=cv.imread("NewSizedImageGray.jpg")
n_m, n_0 = compare_images(img1, img2)
print("Photo norm:", n_m, "/ per pixel:", n_m/img1.size)
print("Zero norm:", n_0, "/ per pixel:", n_0*1.0/img1.size)

if(compareimagepercntage(n_m,img1)):
  print(n_m/img1.size*100)
  print("Images are of the Same person")
else:
  print(n_m/img1.size*100)
  print("Images Are of Different People")

time2=time.time()
par_time=Time.timediffpar(time2,time1)
print("Time taken for parallel program is ",par_time)


2


<IPython.core.display.Javascript object>

Saved to 123.jpg
(480, 640)


<IPython.core.display.Javascript object>

Saved to 2nd.jpg
(480, 640)
Photo norm: 24889.701960784314 / per pixel: 0.1338134436583513
Zero norm: 175549.0 / per pixel: 0.9437966054310952
13.38134436583513
Images are of the Same person
Time taken for parallel program is  0.08734381385585352


**Serial Execution**

In [ ]:
import concurrent.futures
import Time   
import time
import cv2 as cv
import multiprocessing
import numpy as np
from numpy import array
from scipy import misc
import datetime
import sys
from scipy.misc import imread
from scipy.linalg import norm
from scipy import sum, average
num_cores = multiprocessing.cpu_count()



filename = take_photo("123.jpg")
print('Saved to {}'.format(filename))
img = cv.imread('123.jpg')

face = misc.imread('123.jpg', cv.IMREAD_UNCHANGED)
print (face.shape)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('123.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
(x, y, w, h)=(0,0,0,0)
for (x, y, w, h) in faces:
    cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('facecircledimg.jpg', img)

cv.imwrite('face.jpg', frame)

time1=time.time()

img = cv.imread("face.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
for i in range(H):
    for j in range(W):
        gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('facegray.jpg', gray)



filename = take_photo("2nd.jpg")
print('Saved to {}'.format(filename))
img = cv.imread('2nd.jpg')
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('2nd.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
for (x, y, w, h) in faces:
    cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('Face2.jpg', frame)
cv.imwrite('facecircledimg1.jpg', img)


img = cv.imread("Face2.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
for i in range(H):
    for j in range(W):
        gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('Facegray2.jpg', gray)
cv.waitKey()

def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

def compare_images(img1, img2):
    img1 = normalize(img1)
    img2 = normalize(img2)
    diff = img1 - img2 
    m_norm = sum(abs(diff))
    z_norm = norm(diff.ravel(), 0) 
    return (m_norm, z_norm)

img1=cv.imread("Facegray2.jpg")
img2=cv.imread("facegray.jpg")
x=len(np.array(img1))
y=len(np.array(img2))

width = int(img1.shape[1]*y/x)
height = int(img1.shape[0]*y/x)
dim = (width, height)
# resize image
resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
cv.imwrite('NewSizedImageGray.jpg',resized) 

img1=cv.imread("NewSizedImageGray.jpg")
n_m, n_0 = compare_images(img1, img2)
print("Photo norm:", n_m, "/ per pixel:", n_m/img1.size)
print("Zero norm:", n_0, "/ per pixel:", n_0*1.0/img1.size)

if(n_m/img1.size*100<25):
    print("Images Are of same Person!!!!!Hurray...!!!")
else:
    print("Images Are of Different People")

time2=time.time()
ser_time=Time.timediffparsr(time2,time1)
print("Time For Serial Execution - ",ser_time)


<IPython.core.display.Javascript object>

Saved to 123.jpg
(480, 640)


<IPython.core.display.Javascript object>

Saved to 2nd.jpg
Photo norm: 21643.160784313724 / per pixel: 0.12953150905100141
Zero norm: 157824.0 / per pixel: 0.9445561620224073
Images Are of same Person!!!!!Hurray...!!!
Time For Serial Execution -  0.7796968967109902


In [ ]:
print("Speedup Ratio for the program is",ser_time/par_time)

Speedup Ratio for the program is 8.926755797471252


**Compare 2 photos**

Serial

In [6]:
import concurrent.futures
import Time   
import time
import cv2 as cv
import multiprocessing
import numpy as np
from numpy import array
from scipy import misc
import datetime
import sys
from scipy.misc import imread
from scipy.linalg import norm
from scipy import sum, average
import warnings
warnings.simplefilter("ignore")
num_cores = multiprocessing.cpu_count()

face = misc.imread('photo1.jpg', cv.IMREAD_UNCHANGED)
print (face.shape)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('photo1.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
(x, y, w, h)=(0,0,0,0)
for (x, y, w, h) in faces:
    cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('facecircledimg.jpg', img)

cv.imwrite('Face.jpg', frame)

time1=time.time()

img = cv.imread("Face.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
for i in range(H):
    for j in range(W):
        gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('Facegray.jpg', gray)



face = misc.imread('photo2.jpg', cv.IMREAD_UNCHANGED)
print (face.shape)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('photo2.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
for (x, y, w, h) in faces:
    cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('Face2.jpg', frame)
cv.imwrite('facecircledimg1.jpg', img)


img = cv.imread("Face2.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
for i in range(H):
    for j in range(W):
        gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('Facegray2.jpg', gray)
cv.waitKey()

def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

def compare_images(img1, img2):
    img1 = normalize(img1)
    img2 = normalize(img2)
    diff = img1 - img2 
    m_norm = sum(abs(diff))
    z_norm = norm(diff.ravel(), 0) 
    return (m_norm, z_norm)

img1=cv.imread("Facegray2.jpg")
img2=cv.imread("Facegray.jpg")
x=len(np.array(img1))
y=len(np.array(img2))

width = int(img1.shape[1]*y/x)
height = int(img1.shape[0]*y/x)
dim = (width, height)
# resize image
resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
cv.imwrite('NewSizedImageGray.jpg',resized) 

img1=cv.imread("NewSizedImageGray.jpg")
n_m, n_0 = compare_images(img1, img2)
print("Photo norm:", n_m, "/ per pixel:", n_m/img1.size)
print("Zero norm:", n_0, "/ per pixel:", n_0*1.0/img1.size)

print(n_m/img1.size*100)
if(n_m/img1.size*100<27.5):
    print("Images Are of same Person!")
else:
    print("Images Are of Different People")

time2=time.time()
ser_time=Time.timediffparsr(time2,time1)
print("Time For Serial Execution - ",ser_time)

(600, 500)
(847, 696)
Photo norm: 85799.36754759874 / per pixel: 0.31566747809111284
Zero norm: 271797.0 / per pixel: 0.9999779251884637
31.566747809111284
Images Are of Different People
Time For Serial Execution -  0.7719303457472553


Parallel

In [7]:
import concurrent.futures
import Time   
import time
import cv2 as cv
import multiprocessing
import numpy as np
from numpy import array
from scipy import misc
import datetime
import sys
from scipy.misc import imread
from scipy.linalg import norm
from scipy import sum, average
num_cores = multiprocessing.cpu_count()
print(num_cores)
import warnings
warnings.simplefilter("ignore")

face = misc.imread('photo1.jpg', cv.IMREAD_UNCHANGED)
print (face.shape)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('photo1.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
(x, y, w, h)=(0,0,0,0)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for (x, y, w, h) in faces:
        cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('facecircledimg.jpg', img)

cv.imwrite('face.jpg', frame)

time1=time.time()

img = cv.imread("face.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(H):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            for j in range(W):
                gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('Facegray.jpg', gray)

face = misc.imread('photo2.jpg', cv.IMREAD_UNCHANGED)
print (face.shape)
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
img = cv.imread('photo2.jpg')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for (x, y, w, h) in faces:
        cv.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
frame = img[y:y+h, x:x+w]
cv.imwrite('Face2.jpg', frame)
cv.imwrite('facecircledimg1.jpg', img)


img = cv.imread("Face2.jpg")
H,W = img.shape[:2]
gray = np.zeros((H,W), np.uint8)
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i in range(H):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            for j in range(W):
                gray[i,j] = np.clip(0.07 * img[i,j,0]  + 0.72 * img[i,j,1] + 0.21 * img[i,j,2], 0, 255)
cv.imwrite('Facegray2.jpg', gray)
cv.waitKey()

def normalize(arr):
    rng = arr.max()-arr.min()
    amin = arr.min()
    return (arr-amin)*255/rng

def compare_images(img1, img2):
    img1 = normalize(img1)
    img2 = normalize(img2)
    diff = img1 - img2 
    m_norm = sum(abs(diff))
    z_norm = norm(diff.ravel(), 0) 
    return (m_norm, z_norm)

img1=cv.imread("Facegray2.jpg")
img2=cv.imread("Facegray.jpg")
x=len(np.array(img1))
y=len(np.array(img2))

width = int(img1.shape[1]*y/x)
height = int(img1.shape[0]*y/x)
dim = (width, height)
# resize image
resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
cv.imwrite('NewSizedImageGray.jpg',resized) 

img1=cv.imread("NewSizedImageGray.jpg")
n_m, n_0 = compare_images(img1, img2)
print("Photo norm:", n_m, "/ per pixel:", n_m/img1.size)
print("Zero norm:", n_0, "/ per pixel:", n_0*1.0/img1.size)

print(n_m/img1.size*100)
if(n_m/img1.size*100<27.5):
    print("Images Are of same Person!")
else:
    print("Images are of Different People")

time2=time.time()
par_time=Time.timediffparnm(time2,time1)
print("Time For parallel Execution - ",par_time)

2
(600, 500)
(847, 696)
Photo norm: 85799.36754759874 / per pixel: 0.31566747809111284
Zero norm: 271797.0 / per pixel: 0.9999779251884637
31.566747809111284
Images are of Different People
Time For parallel Execution -  0.21193034574725528


In [8]:
print("Speedup Ratio for the program is",ser_time/par_time)

Speedup Ratio for the program is 3.6423776077251673
